# 03 - 统计回顾：最危险的公式

在 2007 年的著名文章中，霍华德·韦纳 (Howard Wainer) 针对非常危险的公式做了如下说明：

“有些公式如果你知道它们的话就很危险，而另一些公式如果你不知道它们就会很危险。第一类之所以可能会带来危险，是因为这些公式揭示了通往背后隐藏着的可怕危险。这其中最著名的是爱因斯坦相对论的经典公式：\\\ (E = MC^2\\)，因为它揭开了隐藏在普通物质中的巨大能量。... 另一方面，我对那些当我们不知道的时候会带来危险，而当我们明白了道理后反而没有问题的公式更感兴趣。当我们熟悉这些公式的时候，它们能让我们能够清楚地理解事物，但一旦不了解就会让我们处在无知的危险中。”

他所说的就是 Moivre 公式：

$
SE = \dfrac{\sigma}{\sqrt{n}}
$

其中 \\(SE\\) 是平均值的标准误差，\\(\sigma\\) 是标准偏差，\\(n\\) 是样本大小。听起来像一个勇敢而真实的人应该掌握的数学，所以让我们开始认真学习它。

要了解为什么不知道这个等式是非常危险的，让我们来看看一些教育数据。我已经收集了来自不同学校的 ENEM 分数（巴西标准化高中分数，类似于 SAT）的数据，为期 3 年。我还对数据进行了一些清理，只保留与我们相关的信息。原始数据可在[Inep网站]下载。(http://portal.inep.gov.br/web/guest/microdados#)下载。

如果我们看看表现最好的学校，就会注意到一些事情：这些学校的学生人数相当少。

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import style
style.use("fivethirtyeight")

In [2]:
df = pd.read_csv("./data/enem_scores.csv")
df.sort_values(by="avg_score", ascending=False).head(10)

,year,school_id,number_of_students,avg_score
16670,2007,33062633,68,82.97
16796,2007,33065403,172,82.04
16668,2005,33062633,59,81.89
16794,2005,33065403,177,81.66
10043,2007,29342880,43,80.32
18121,2007,33152314,14,79.82
16781,2007,33065250,80,79.67
3026,2007,22025740,144,79.52
14636,2007,31311723,222,79.41
17318,2007,33087679,210,79.38
